In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Data from Global Solar Atlas (Tilt: 40°, Azimuth: 180°, 8000kW)
G_atlas = 1205.9 # kW/m2
P_atlas = 7.954E6 # kWh
A_atlas = P_atlas/G_atlas # m2

# Data from renewables.ninja (Tilt: 40°, Azimuth: 180°, 8000kW)
pv_ninja = pd.read_csv('data/solar/ninja_pv_56.4000_6.8000_uncorrected.csv',
                       sep = ',',
                       index_col = 0,
                       usecols = [0,2,3,4],
                       comment = '#')

In [3]:
P = pv_ninja['electricity'].values/pv_ninja['electricity'].mean()*P_atlas/8760 # kWh
cf_pv = P/8000
cf_pv.mean()

0.11349885844748858

In [4]:
pd.Series(cf_pv).to_csv('data/cf/pv.csv')